Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train and explain models remotely via Azure Machine Learning Compute
## Introduction

This notebook showcases how to train and explain a model remotely via Azure Machine Learning Compute (AMLCompute), and download the calculated explanations locally for visualization.
It demonstrates the API calls that you need to make to submit a run for training and explaining a model to AMLCompute, download the compute explanations remotely, and visualizing the global and local explanations via a visualization dashboard that provides an interactive way of discovering patterns in model predictions and downloaded explanations.

We will showcase one of the tabular data explainers: TabularExplainer (SHAP).

Problem: German credit risk Prediction with scikit-learn (train a model and run an explainer remotely via AMLCompute, and download and visualize the remotely-calculated explanations.)

![explanations-run-history](./img/explanations-run-history.png)

## Setup

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize a Workspace

Connect to the workspace

In [ ]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep='\n')

In [ ]:
# get the German credit dataset from the workspace
creditData = ws.datasets['german-credit'].to_pandas_dataframe()
creditData["Risk_cat"] = creditData["Risk"].astype('category').cat.codes
creditData.head()

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'explainer-remote-run-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.



## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil

project_folder = './explainer-remote-run-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain.py', project_folder)

### Fetch or create the compute target 

We are going to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`. 

In [ ]:
# Define remote compute target to use
# Further docs on Remote Compute Target: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-remote
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 idle_seconds_before_scaledown=10800,
                                                                 min_nodes=0,
                                                                 max_nodes = 20)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')

# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

### Configure & Run

In [ ]:
from azureml.train.estimator import Estimator

pip_packages = [
                'azureml-defaults==1.0.76', 'azureml-core==1.0.76', 'azureml-telemetry==1.0.76',
                'azureml-dataprep==1.1.31', 'joblib==0.14.0', 'sklearn-pandas==1.7.0', 'pandas==0.23.4',
                'azureml-contrib-interpret'
               ]

estimator = Estimator(source_directory=project_folder, 
                      compute_target=aml_remote_compute,
                      entry_script='train_explain.py',
                      pip_packages=pip_packages,
                      conda_packages=['scikit-learn==0.22.2.post1'], #['scikit-learn==0.20.3']
                      inputs=[ws.datasets['german-credit'].as_named_input('credit')])

run = experiment.submit(estimator)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

![](img/aml-run.png)

## (OPTIONAL) Additional operations to perform on AmlCompute Cluster

You can perform more operations on AmlCompute such as updating the node counts or deleting the compute. 

In [ ]:
# Get_status () gets the latest status of the AmlCompute target
#aml_remote_compute.get_status().serialize()

In [ ]:
# Update () takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
# aml_remote_compute.update(min_nodes=1)
# aml_remote_compute.update(max_nodes=10)
# aml_remote_compute.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)
#aml_remote_compute.update(idle_seconds_before_scaledown=10800)

In [ ]:
# Delete () is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
# 'aml_remote_compute' in this case but use a different VM family for instance.
# aml_remote_compute.delete()

In [ ]:
run.wait_for_completion(show_output=True)

## Download 
### 1. Download model explanation data.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()

In [ ]:
# Get the top k (e.g., 5) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=5)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

#### See model's files

In [ ]:
print(run.get_file_names())

### 2. Download model and test set files

In [ ]:
# retrieve model for visualization and deployment
from azureml.core.model import Model
import joblib

# Download test dataset file
run.download_file('x_test_credit.pkl')
x_test = joblib.load('x_test_credit.pkl')

# Download trained model
run.download_file('model.pkl')
original_model = joblib.load('model.pkl')

original_model

## Visualize
Load the visualization dashboard (you can also use AML Studio dashboard)

In [ ]:
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(global_explanation, original_model, datasetX=x_test)